In [2]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

In [3]:
data

Data(edge_index=[2, 4], x=[3, 1])

In [7]:
print(data.keys)

['x', 'edge_index']


In [8]:
print(data['x'])

tensor([[-1.],
        [ 0.],
        [ 1.]])


In [9]:
for key, item in data:
    print("{} found in data".format(key))

edge_index found in data
x found in data


In [10]:
'edge_attr' in data

False

In [11]:
print(data.num_nodes)
print(data.num_edges)
print(data.num_node_features)
print(data.contains_isolated_nodes())
print(data.contains_self_loops())
print(data.is_directed())

3
4
1
False
False
False


In [12]:
# Transfer data object to GPU.
device = torch.device('cuda')
data = data.to(device)

## Enzyme dataset: each data corresponds to class, not node

In [20]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset

ENZYMES(600)

In [21]:
print(len(dataset))
print(dataset.num_classes) # kinds of y
print(dataset.num_node_features) # num of node features

600
6
3


In [22]:
dataset[0]

Data(edge_index=[2, 168], x=[37, 3], y=[1])

In [23]:
data.is_undirected()

True

## Cora dataset: each node in data corresponds to class, not data

In [24]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Cora()

In [25]:
print(len(dataset))
print(dataset.num_classes) # kinds of y
print(dataset.num_node_features) # num of node features

1
7
1433


In [26]:
dataset[0]

Data(edge_index=[2, 10556], test_mask=[2708], train_mask=[2708], val_mask=[2708], x=[2708, 1433], y=[2708])

## Mini-batch

In [32]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
from torch_scatter import scatter_mean

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

for data in loader:
    print(data)
    print(data.num_graphs)
    print(data.batch)
    x = scatter_mean(data.x, data.batch, dim=0)
    print(x.size())

Batch(batch=[1043], edge_index=[2, 3912], x=[1043, 21], y=[32])
32
tensor([ 0,  0,  0,  ..., 31, 31, 31])
torch.Size([32, 21])
Batch(batch=[1051], edge_index=[2, 4020], x=[1051, 21], y=[32])
32
tensor([ 0,  0,  0,  ..., 31, 31, 31])
torch.Size([32, 21])
Batch(batch=[1092], edge_index=[2, 3976], x=[1092, 21], y=[32])
32
tensor([ 0,  0,  0,  ..., 31, 31, 31])
torch.Size([32, 21])
Batch(batch=[1065], edge_index=[2, 4014], x=[1065, 21], y=[32])
32
tensor([ 0,  0,  0,  ..., 31, 31, 31])
torch.Size([32, 21])
Batch(batch=[944], edge_index=[2, 3654], x=[944, 21], y=[32])
32
tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3,  3,  4,  4,

## Transform

In [33]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'])

dataset[0]

Extracting /tmp/ShapeNet/shapenetcore_partanno_segmentation_benchmark_v0_normal.zip
Processing...
Done!


Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])

In [34]:
import torch_geometric.transforms as T
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6))

dataset[0]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch_geometric/data/dataset.py:151: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you really want to make use of another pre-processing technique, make sure to delete `/tmp/ShapeNet/processed` first.
  'sure to delete `{}` first.'.format(self.processed_dir))


Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])

In [35]:
from torch_geometric.datasets import ShapeNet

dataset = ShapeNet(root='/tmp/ShapeNet', categories=['Airplane'],
                    pre_transform=T.KNNGraph(k=6),
                    transform=T.RandomTranslate(0.01))

dataset[0]

/home/ubuntu/anaconda3/lib/python3.6/site-packages/torch_geometric/data/dataset.py:151: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you really want to make use of another pre-processing technique, make sure to delete `/tmp/ShapeNet/processed` first.
  'sure to delete `{}` first.'.format(self.processed_dir))


Data(category=[1], pos=[2518, 3], x=[2518, 3], y=[2518])

In [36]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Cora()

In [37]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

In [40]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.7980
